# Test yours models

Test your models, trained with Tensorflow Keras and Spacy.

Model are loaded from models directory.

## Setup

In [1]:
from ipywidgets import interact_manual
from ipywidgets import widgets

import re
import string
import pickle
import numpy as np
import spacy
import os

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from spacy import displacy

layout = widgets.Layout(width='auto', height='100px') #set width and height

## Ner with Bi-directional LSTM

In [2]:
model = load_model("../models/bilstm.h5")

# Saving Vocab
with open('../models/word_to_index.pickle', 'rb') as handle:
    word2idx = pickle.load(handle)
    
# Saving Vocab
with open('../models/index_to_tag.pickle', 'rb') as handle:
    idx2tag = pickle.load(handle)
    
MAX_LEN = 50

In [3]:
from ipywidgets import interact_manual
from ipywidgets import widgets

import re
import string

layout = widgets.Layout(width='auto', height='100px') #set width and height

# Custom Tokenizer
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
    
def get_prediction(sentence):
    test_sentence = tokenize(sentence) # Tokenization
    # Preprocessing
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=word2idx["PAD"], maxlen=MAX_LEN)
    # Evaluation
    p = model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    # Visualization
    print("{:15}||{}".format("Word", "Prediction"))
    print(30 * "=")
    for w, pred in zip(test_sentence, p[0]):
        print("{:15}: {:5}".format(w, idx2tag[pred]))

interact_manual(get_prediction, sentence=widgets.Textarea(placeholder='Type your sentence here', layout=layout));

interactive(children=(Textarea(value='', description='sentence', layout=Layout(height='100px', width='auto'), …

## Ner with Spacy

In [4]:
new_model_name="ner_model" 
output_dir="../models/ner_model"

In [5]:
# test the saved model
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

Loading from ../models/ner_model


In [6]:
sentence = "The 1906 San Francisco earthquake was the biggest earthquake that has ever hit San Francisco on April 18, 1906"

In [7]:
doc2 = nlp(sentence)
displacy.render(doc2, style="ent")